## Packages & functions

In [1]:
# Author: Antti Kiviaho
# Date: 28.8.2023

import os 
os.chdir('/lustre/scratch/kiviaho/prostate_spatial')
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

import random

import seaborn as sns
import matplotlib.pyplot as plt


from scipy.stats import ttest_1samp, chisquare
from statsmodels.stats.multitest import multipletests
from itertools import combinations
from datetime import datetime

from scripts.utils import load_from_pickle
import warnings
warnings.filterwarnings("ignore")


In [2]:
def robust_nmf_programs_within_sample(dict_obj,jaccard_threshold = 0.7):

    res_df = pd.DataFrame()
    for k in list(dict_obj.keys()):
        cols = []
        df = dict_obj[k]
        for c in list(combinations(df.columns,2)):
            if jaccard(df[c[0]],df[c[1]])>jaccard_threshold:
                cols.extend([c[0],c[1]])
            
        df = df[set(cols)]
        df.columns = [k+'.'+name for name in df.columns]
        res_df = pd.concat([res_df,df],axis=1)
    
    return(res_df)


def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

def robust_nmf_programs_between_samples(flattened_df,jaccard_threshold = 0.2):

    cols_to_keep = []
    # Take a column one at a time
    for col, lst in flattened_df.iteritems():
        
        # Remove nmf programs from the same sample before comparing
        rest_df = flattened_df.loc[:,~flattened_df.columns.str.contains(col.split('.')[0])]

        # Calculate jaccard index for each program in other samples
        for column_name, column_data in rest_df.iteritems():
            if jaccard(lst,column_data) > jaccard_threshold:
                cols_to_keep.extend([col,column_name])
    
    return(flattened_df[set(cols_to_keep)])


def create_jaccard_interaction_matrix(df,threshold_value=0.2):

    # Initialize the interaction matrix
    interaction_matrix = np.zeros((df.shape[1], df.shape[1]))

    # Iterate over the columns
    for i, (_, col1) in enumerate(df.iteritems()):
        for j, (_, col2) in enumerate(df.iteritems()):
            # Calculate the Jaccard index
            jac_index = jaccard(col1, col2)

            # Save the Jaccard index in the interaction matrix
            interaction_matrix[i, j] = jac_index

    df_res = pd.DataFrame(interaction_matrix,index=df.columns,columns=df.columns)

    df_res[df_res < threshold_value] = 0

    return(df_res)


def drop_redundant_programs(df, jaccard_threshold):
    
    # Step 1: Rank the rows according to their max values and iterate starting from the highest
    for row_idx in df.max(axis=1).rank(ascending=False).sort_values().index:

        to_drop = [] # list structure for redundant columns

        # Find which column has the max value
        col_name = df.loc[row_idx].idxmax()

        # Iterate over all the rest
        cols = [c for c in df.columns if c != col_name]
        for c in cols:
            
            # Add columns with overlapping jaccard (>0.2) to be dropped
            if df.at[row_idx,c] >= jaccard_threshold:
                to_drop.append(c)

        # Drop the column before the next iteration
        df = df.drop(columns=to_drop)

    return(df)

def create_program_clusters_from_jaccard_df(df,minimum_overlap=3):
    clusters = {}
    sufficient_overlap = True
    i = 0

    while sufficient_overlap:

        # Rank the rows according to how many sufficient jaccard index overlaps have been observed
        top_overlapping_programs = (df != 0).astype(int).sum(axis=1).sort_values(ascending=False)

        # Continue if minimum overlap threshold is satisfied
        if top_overlapping_programs[0] >= minimum_overlap-1:
            
            # Get the program with most overlaps
            top_program = top_overlapping_programs.index[0]

            # Get programs which overlap with the top one
            programs = list(df[df.loc[top_program] !=0].index)

            programs.append(top_program) # Put them to a single list

            clusters['cluster_'+str(i)] = programs # Save the list

            # Drop programs that have been included already
            df = df.drop(columns=programs)
            df = df.loc[df.columns]

        else:
            sufficient_overlap = False

        i+=1
    
    return(clusters)

def filter_jaccard_interaction_matrix(df,jac_thershold=0.2):

    # Dataframe for result concatenation
    df_final=pd.DataFrame()

    # Get a list of unique samples in the interactions dataframe
    samples = np.unique([c.split('.')[0] +'.' for c in df.columns]).tolist()

    # Check that sample order does not affect end result
    random.shuffle(samples)

    for s in samples:

        # Keep columns that are from sample s, drop rows that are from sample s
        dummy_df = df.filter(like=s).loc[~df.index.str.contains(s)]
        
        # Drop rows that are all zeros (most rows)
        dummy_df = dummy_df.loc[~(dummy_df.sum(axis=1)==0)]

        dummy_df = drop_redundant_programs(dummy_df,jac_thershold)

        df_final = pd.merge(df_final,dummy_df,left_index=True,right_index=True,how='outer')

    # Keep the non redundants (those kept in columns)
    df_final = df_final.loc[df_final.columns]

    df_final.fillna(0,inplace=True)

    return(df_final)


def t_test_score_preprocessing(dat):

    sc.pp.filter_genes(dat,min_counts=10)
    sc.pp.normalize_total(dat)
    sc.pp.scale(dat)

    return(dat)


def score_modules_ttest_1samp(dat, genes_dict,p_val_thr=0.05,min_cells=100):

    mods = list(genes_dict.keys())

    adata_lst = []

    n_genes_df = pd.DataFrame()

    # Perform scaling on each sample separately (as done in nmf)
    val_counts = dat.obs['sample'].value_counts()

    for s in val_counts.index:

        if val_counts[s] >= min_cells:

            # Subset the data to only include a single sample
            sub_dat = dat[dat.obs['sample'] == s]

            sub_dat = t_test_score_preprocessing(sub_dat)

            n_genes_remain = []

            # Loop over the gene modules
            for k in genes_dict.keys():

                lst = genes_dict[k]

                # Subset the data to just the genes in question
                subs_X = sub_dat.X[:, np.where(sub_dat.var_names.isin(lst))[0]]
                
                # Save the information about how many genes are used to score this program
                n_genes_remain.append(subs_X.shape[1])

                # One-sample t-test to see whether the gene set is active
                t, p_vals = ttest_1samp(subs_X, axis=1, popmean=0, alternative='greater')

                # Save the p-values into the original dat anndata observations.
                sub_dat.obs[k] = p_vals

            adata_lst.append(sub_dat)

            n_genes_df = pd.concat([n_genes_df,pd.DataFrame({s:n_genes_remain})],axis=1)

    # Concatenate the sample-wise scaled data
    dat = ad.concat(adata_lst)

    # Perform p-value correction
    dat_obs = dat.obs[mods]

    # Perform Benjamini-Hochberg correction on each column of the DataFrame
    for column in dat_obs.columns:
        pvals = dat_obs[column].values
        reject, corr_pvals, _, _ = multipletests(pvals, method='fdr_bh')
        dat.obs[column] = corr_pvals

    # Take each column with an adjusted p_value < 0.05 and concatenate the modules into a string and save it row-wise (annotation)
    dat.obs['annotation'] = (dat.obs[mods] < p_val_thr).astype(int).multiply(dat.obs[mods].columns.str.replace('cluster','')).sum(axis=1)

    n_genes_df.index = mods
    n_genes_df = n_genes_df.T

    return(dat.obs,n_genes_df)


def run_chi2_test(df,annot_col='annotation',comparison_col='phenotype'):

    # Phenotype proportions in the celltype as a whole
    bground_proportions = df[comparison_col].value_counts()/len(df)
    order = bground_proportions.index

    for module in df[annot_col].value_counts().index:
        # The number of cells scoring highest on this module
        subset = df[df[annot_col] == module]

        # Expected proportions based on the celltype proportions
        expected = bground_proportions*len(subset)

        try: # A lazy solution to some categories missing 
            observed = subset[comparison_col].value_counts()[order]

            contig_table = pd.DataFrame({'expected':expected.astype(int),
            'observed':observed,
            'diff':observed-expected.astype(int)})

            pval = chisquare(observed,expected).pvalue

            print(module+': top scoring in ' + str(len(subset))+' cells')
            print(contig_table)
            print('chi2 test p-value: ' + str(pval))
            print('')
        except:
            print(module + ' is missing a category: ')
            print(subset[comparison_col].value_counts())
            continue


def save_dict_to_excel(dict_with_modules,ctype):
    # Saves the factors to a dataframe and onto an excel sheet
    df_with_modules = pd.DataFrame.from_dict(dict_with_modules)
    df_with_modules.to_excel(ctype+'_gavish_mode_modules_'+datetime.today().strftime('%Y%m%d')+'.xlsx')

def subset_modules(dictionary,length):
    for k in dictionary.keys():
        dictionary[k] = dictionary[k].iloc[:length]

    return(dictionary)


## Running analyses

In [3]:
# Create a dataframe for final annotations of all cells
final_annotations = pd.DataFrame()

# Parameter configurations
intra_overlap = 0.5
inter_overlap = 0.3
min_samples = 3

### Immune cells

In [4]:
immune_cell_annotations = pd.read_csv('./celltypist_dir/celltypist_predicted_labels_20230824.csv',index_col=0)
immune_cell_annotations['predicted_labels']

final_annotations = pd.concat([final_annotations,immune_cell_annotations['predicted_labels']],axis=0)

### Epithelials

In [5]:
filter_phenotype = ''
celltype = 'Epithelial'

adata = sc.read_h5ad('./nmf_annotation/'+celltype+'.h5ad')

# Extract the counts
adata.X = adata.layers['counts'].copy()

if filter_phenotype != '':
    adata = adata[adata.obs['phenotype'] == filter_phenotype]

adata_obs = adata.obs.copy()

In [6]:
# modules is a dictionary with samples as keys and 50 row by 39 col dataframes as entries (top NMF contributors)
modules = load_from_pickle('./nmf_annotation/'+celltype+'_top_50_genes_for_nmf_components_4_to_9.pkl')
#modules = subset_modules(modules,30)

# Subset modules according to a phenotype (Epithelials)
if filter_phenotype != '':
    modules = {k: v for k, v in modules.items() if k in list(adata_obs[adata_obs['phenotype'] == filter_phenotype]['sample'].unique())}

# 1st filter
# Find programs that are robust within samples (returns df)
robust_nmf_within = robust_nmf_programs_within_sample(modules,intra_overlap)


# 2nd filter
# Find programs that are robust across samples (returns df)
robust_nmf_within_and_between =  robust_nmf_programs_between_samples(robust_nmf_within,inter_overlap) # Change this in accordance


# 3rd filter
# Get rid of programs that are redundant within samples (returns df)
jaccards_with_redundants = create_jaccard_interaction_matrix(robust_nmf_within_and_between)
robust_nmf_program_jaccards = filter_jaccard_interaction_matrix(jaccards_with_redundants,inter_overlap) # With this
robust_nmf_programs = robust_nmf_within_and_between[robust_nmf_program_jaccards.columns]


# Put together ovelapping programs
clusters = create_program_clusters_from_jaccard_df(robust_nmf_program_jaccards,minimum_overlap = min_samples)

In [11]:
gene_modules = {}
for k in clusters.keys():

    # Take the top 50 genes in a cluster. These are the final ones
    genes = list(robust_nmf_programs[clusters[k]].stack().reset_index(drop=True).value_counts()[:50].index)

    # Added checks to not add overlapping modules (>0.1 jaccard index with an existing module)
    if len(gene_modules) > 0:
        jaccards = []
        for k2 in gene_modules.keys():
            jaccards.append(jaccard(genes,gene_modules[k2]))
        if max(jaccards) < 0.1:
            gene_modules[k] = genes
    else:
        # There aren't yet modules
        gene_modules[k] = genes
        continue

pd.DataFrame(gene_modules)

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12
0,RDH11,CSRNP1,CKS1B,ANXA2,MT2A,CTSB,S100P,PERP,IL7R,RPL10A,IFI27,ISG20
1,DBI,IER2,HMGB2,ZFP36L1,ISG20,KRT5,SCCPDH,GLUL,SLC2A3,RPS18,MEG8,ST14
2,KLK2,PPP1R15A,CENPW,S100A6,MT1E,PIK3R1,PSCA,CSTA,PTPRCAP,RPS5,NRP2,KRT13
3,SEC11C,FOSB,SMC4,GSTP1,CCK,F3,SMIM5,GPX2,RGS1,RPS3,MEG3,EMP1
4,DHRS7,FOS,MAD2L1,CD74,TM4SF1,DST,UCP2,ANXA8,CYTIP,RPS2,ASCL1,ABHD11
5,STEAP2,JUNB,PTTG1,THSD4,CAVIN1,SLC14A1,ST3GAL4,TMEM40,CD3E,RPS11,GRP,DAPP1
6,PMEPA1,SERTAD1,TUBA1B,NTN4,SOD2,TXNIP,CXCL17,LAMB3,CD7,RPLP0,SYT1,GPRC5A
7,KLK3,MAFF,CKS2,TACSTD2,SOCS2,PTPN14,TRIM31,NSG1,PTPRC,RPS24,TUBB2B,ADGRF1
8,CPE,BTG2,UBE2T,ETS2,EMP1,THSD4,MUC1,KRT15,ZNF331,RPS14,ARL4C,TMPRSS4
9,KLK4,CCNL1,UBE2S,ATP1B1,CEACAM1,TNS1,OAS1,PCP4L1,S100A4,RPL31,CEACAM6,CRABP2


In [12]:
gene_modules.pop('cluster_10')
save_dict_to_excel(gene_modules,celltype)

In [13]:
score_obs, genes_used_meta_df = score_modules_ttest_1samp(adata,gene_modules)

# Subset those that were significant for just one cluster
scores_epi_final = score_obs[score_obs['annotation'].str.count('_') == 1]
scores_epi_final['annotation'] = celltype + scores_epi_final['annotation']

In [14]:
run_chi2_test(scores_epi_final,annot_col='annotation',comparison_col='phenotype')

Epithelial_1: top scoring in 10501 cells
        expected  observed  diff
PCa         5923      6834   911
normal      2689      1930  -759
CRPC        1887      1737  -150
chi2 test p-value: 2.7848256756177434e-80

Epithelial_0: top scoring in 9247 cells
        expected  observed  diff
PCa         5216      5750   534
normal      2368      3066   698
CRPC        1662       431 -1231
chi2 test p-value: 3.0185417804027107e-255

Epithelial_3: top scoring in 2443 cells
        expected  observed  diff
PCa         1378      1524   146
normal       625       408  -217
CRPC         439       511    72
chi2 test p-value: 4.4461939200953393e-23

Epithelial_2: top scoring in 2042 cells
        expected  observed  diff
PCa         1151       434  -717
normal       522        94  -428
CRPC         367      1514  1147
chi2 test p-value: 0.0

Epithelial_12: top scoring in 1782 cells
        expected  observed  diff
PCa         1005       182  -823
normal       456      1495  1039
CRPC         320 

In [15]:
final_annotations = pd.concat([final_annotations,scores_epi_final['annotation']],axis=0)

## Fibroblast & muscle

In [16]:
celltype = 'Fibroblast_muscle'

adata = sc.read_h5ad('./nmf_annotation/'+celltype+'.h5ad')

# Extract the counts
adata.X = adata.layers['counts'].copy()

adata_obs = adata.obs.copy()

In [17]:
# modules is a dictionary with samples as keys and 50 row by 39 col dataframes as entries (top NMF contributors)
modules = load_from_pickle('./nmf_annotation/'+celltype+'_top_50_genes_for_nmf_components_4_to_9.pkl')
#modules = subset_modules(modules,30)

# 1st filter
# Find programs that are robust within samples (returns df)
robust_nmf_within = robust_nmf_programs_within_sample(modules,intra_overlap)


# 2nd filter
# Find programs that are robust across samples (returns df)
robust_nmf_within_and_between =  robust_nmf_programs_between_samples(robust_nmf_within,inter_overlap) # Change this in accordance


# 3rd filter
# Get rid of programs that are redundant within samples (returns df)
jaccards_with_redundants = create_jaccard_interaction_matrix(robust_nmf_within_and_between)
robust_nmf_program_jaccards = filter_jaccard_interaction_matrix(jaccards_with_redundants,inter_overlap) # With this
robust_nmf_programs = robust_nmf_within_and_between[robust_nmf_program_jaccards.columns]


# Put together ovelapping programs
clusters = create_program_clusters_from_jaccard_df(robust_nmf_program_jaccards,minimum_overlap=min_samples)

In [18]:
gene_modules = {}
for k in clusters.keys():

    # Take the top 50 genes in a cluster. These are the final ones
    genes = list(robust_nmf_programs[clusters[k]].stack().reset_index(drop=True).value_counts()[:50].index)

    # Added checks to not add overlapping modules (>0.1 jaccard index with an existing module)
    if len(gene_modules) > 0:
        jaccards = []
        for k2 in gene_modules.keys():
            jaccards.append(jaccard(genes,gene_modules[k2]))
        if max(jaccards) < 0.1:
            gene_modules[k] = genes
    else:
        # There aren't yet modules
        gene_modules[k] = genes
        continue

pd.DataFrame(gene_modules)

,cluster_0,cluster_1,cluster_2
0,TPM2,SERPINF1,ELF3
1,MYL9,MMP2,LAMC2
2,ACTA2,ISLR,KRT19
3,TPM1,IGF1,CLDN7
4,DSTN,SPOCK3,KLF5
5,TAGLN,DCN,KRT17
6,MYH11,MFAP2,MAST4
7,FLNA,PODN,KRT8
8,CSRP1,TCF21,WFDC2
9,MYLK,PTN,VAMP8


In [19]:

save_dict_to_excel(gene_modules,celltype)

In [20]:
score_obs, genes_used_meta_df = score_modules_ttest_1samp(adata,gene_modules)

# Subset those that were significant for just one cluster
scores_fibro_final = score_obs[score_obs['annotation'].str.count('_') == 1]
scores_fibro_final['annotation'] = celltype + scores_fibro_final['annotation']

In [21]:
run_chi2_test(scores_fibro_final,annot_col='annotation',comparison_col='phenotype')

Fibroblast_muscle_0: top scoring in 4369 cells
        expected  observed  diff
normal      2927      2838   -89
PCa          946       994    48
CRPC         494       537    43
chi2 test p-value: 0.012445372701320777

Fibroblast_muscle_1: top scoring in 2835 cells
        expected  observed  diff
normal      1899      1710  -189
PCa          614       778   164
CRPC         321       347    26
chi2 test p-value: 8.230712271906815e-15

Fibroblast_muscle_2: top scoring in 1282 cells
        expected  observed  diff
normal       859      1139   280
PCa          277        66  -211
CRPC         145        77   -68
chi2 test p-value: 1.6373861391436925e-62



In [22]:
final_annotations = pd.concat([final_annotations,scores_fibro_final['annotation']],axis=0)

### Endothelial

In [23]:
celltype = 'Endothelial'

adata = sc.read_h5ad('./nmf_annotation/'+celltype+'.h5ad')

# Extract the counts
adata.X = adata.layers['counts'].copy()

adata_obs = adata.obs.copy()

In [24]:
# modules is a dictionary with samples as keys and 50 row by 39 col dataframes as entries (top NMF contributors)
modules = load_from_pickle('./nmf_annotation/'+celltype+'_top_50_genes_for_nmf_components_4_to_9.pkl')
#modules = subset_modules(modules,30)

# 1st filter
# Find programs that are robust within samples (returns df)
robust_nmf_within = robust_nmf_programs_within_sample(modules,intra_overlap)


# 2nd filter
# Find programs that are robust across samples (returns df)
robust_nmf_within_and_between =  robust_nmf_programs_between_samples(robust_nmf_within,inter_overlap) # Change this in accordance


# 3rd filter
# Get rid of programs that are redundant within samples (returns df)
jaccards_with_redundants = create_jaccard_interaction_matrix(robust_nmf_within_and_between)
robust_nmf_program_jaccards = filter_jaccard_interaction_matrix(jaccards_with_redundants,inter_overlap) # With this
robust_nmf_programs = robust_nmf_within_and_between[robust_nmf_program_jaccards.columns]


# Put together ovelapping programs
clusters = create_program_clusters_from_jaccard_df(robust_nmf_program_jaccards,minimum_overlap=min_samples)

In [25]:
gene_modules = {}
for k in clusters.keys():

    # Take the top 50 genes in a cluster. These are the final ones
    genes = list(robust_nmf_programs[clusters[k]].stack().reset_index(drop=True).value_counts()[:50].index)

    # Added checks to not add overlapping modules (>0.1 jaccard index with an existing module)
    if len(gene_modules) > 0:
        jaccards = []
        for k2 in gene_modules.keys():
            jaccards.append(jaccard(genes,gene_modules[k2]))
        if max(jaccards) < 0.1:
            gene_modules[k] = genes
    else:
        # There aren't yet modules
        gene_modules[k] = genes
        continue

pd.DataFrame(gene_modules)

,cluster_0,cluster_1,cluster_2
0,RGS5,SRGN,HSPH1
1,MFGE8,SLC9A3R2,EIF1
2,CALD1,PLLP,SERTAD1
3,MYL9,CLDN5,DYNLL1
4,HIGD1B,ENPP2,UBB
5,TPM2,LTBP4,JUNB
6,NDUFA4L2,TSPAN2,CKS2
7,PDGFRB,ARL15,MAFF
8,CAMK2N1,SRP14,ZFAND2A
9,LGALS1,ICAM2,FOS


In [26]:

save_dict_to_excel(gene_modules,celltype)

In [27]:
score_obs, genes_used_meta_df = score_modules_ttest_1samp(adata,gene_modules)

# Subset those that were significant for just one cluster
scores_endo_final = score_obs[score_obs['annotation'].str.count('_') == 1]
scores_endo_final['annotation'] = celltype + scores_endo_final['annotation']

In [28]:
run_chi2_test(scores_endo_final,annot_col='annotation',comparison_col='phenotype')

Endothelial_2: top scoring in 2490 cells
        expected  observed  diff
PCa         1533      1492   -41
normal       824       899    75
CRPC         132        99   -33
chi2 test p-value: 0.00030384909956046244

Endothelial_1: top scoring in 904 cells
        expected  observed  diff
PCa          556       607    51
normal       299       236   -63
CRPC          48        61    13
chi2 test p-value: 2.1788445391054393e-05

Endothelial_0: top scoring in 372 cells
        expected  observed  diff
PCa          229       220    -9
normal       123       112   -11
CRPC          19        40    21
chi2 test p-value: 1.5744408948007308e-05



In [30]:
final_annotations = pd.concat([final_annotations,scores_endo_final['annotation']],axis=0)

## Merging annotations with data

In [31]:
final_annotations = final_annotations.rename(columns={0:'final_annotation'})

In [32]:
adata = sc.read_h5ad('aggregate_sc_data_with_broad_annotation_20230823.h5ad')
adata.X = adata.layers['counts'].copy()
del adata.layers['counts']

In [33]:
merged_df = pd.merge(adata.obs, final_annotations, left_index=True, right_index=True, how='left')
#merged_df['final_annotation'] = merged_df['final_annotation'].fillna(merged_df['refined_celltypes'])

if (merged_df.index == adata.obs.index).all():
    adata.obs = merged_df.copy()
    print('Done')

# Removing the cells with nan as cell type reference
adata = adata[~adata.obs['final_annotation'].isna()]


Done


In [34]:
# Finished, annotated data
print(adata.obs['final_annotation'].value_counts())
print('')
print('The total number of different celltypes: ' + str(len(adata.obs['final_annotation'].unique())))

T cells                48219
Epithelial_1           10501
Epithelial_0            9247
B cells                 7617
Fibroblast_muscle_0     4369
Mast cells              3594
ILC                     2846
Fibroblast_muscle_1     2835
Endothelial_2           2490
Epithelial_3            2443
Macrophages             2291
Epithelial_2            2042
Epithelial_12           1782
Epithelial_7            1468
Epithelial_4            1407
Fibroblast_muscle_2     1282
Epithelial_8             976
Endothelial_1            904
Epithelial_5             707
Epithelial_9             451
Endothelial_0            372
Epithelial_11            145
Name: final_annotation, dtype: int64

The total number of different celltypes: 22


In [37]:
adata.write('./single_cell_reference_with_gavish_nmf_derived_annotations_'+datetime.today().strftime('%Y%m%d')+'.h5ad')